# Exact Diagonalization of Spin Hamiltonian
    Alan Morningstar
    May 2017

#### specify lattice and Hamiltonian parameters

In [1]:
# NOTE: using periodic boundary conditions
# square lattice length
const Lx = 4;
const Ly = 4;
# number of sites
const N = Lx*Ly;
# NN coupling
const J1 = 1.0;
# NNN coupling
const J2 = 0.0;
# plaquette coupling
const K = 0.0;

In [2]:
# container for Hamiltonian couplings
type couplings
    # 1st nearest neighbor
    J1::Float64;
    # 2nd nearest neighbor
    J2::Float64;
    # plaquette terms
    K::Float64;
end;

In [3]:
const c = couplings(J1,J2,K);

## Lattice and Bond Structure

#### useful functions for defining the lattice connectivity

In [4]:
# convert single integer site index to xy indexing
function xyIndex(site::Int64,Lx::Int64,Ly::Int64)
    # site - number of the site for which x,y indices are computed

    y::Int,x::Int = divrem(site-1,Lx);
    
    return x,y;
end;

In [5]:
# convert xy site indexing to single integer index
function siteIndex(xy::Tuple{Int,Int},Lx::Int64,Ly::Int64)
    # xy - tuple containing x,y indexing of a site

    x = xy[1];
    y = xy[2];
    
    # move xy into primary lattice cell
    while x > (Lx-1)
        x -= Lx;
    end;
    while x < 0
        x += Lx;
    end;
    while y > (Ly-1)
        y -= Ly;
    end;
    while y < 0
        y += Ly;
    end;    
    
    # map to integer index
    site::Int64 = y*Lx + x + 1;
    
    return site;
end;

In [6]:
# find indices of neighbors specified by a translation vectors T
function neighbors(site::Int64,T::Array{Tuple{Int64,Int64},1},Lx::Int64,Ly::Int64)
    # use xy indexing just to compute neighbors
    x::Int,y::Int = xyIndex(site,Lx,Ly);

    # neighbor in xy indexing
    neighborxy::Array{Tuple{Int,Int},1} = [(x+t[1],y+t[2]) for t in T];
    
    # back to single integer indexing
    neighborIndex::Array{Int,1} = [siteIndex(nxy,Lx,Ly) for nxy in neighborxy];
    
    # return neighbor index
    return neighborIndex;
end;

#### define the lattice

In [7]:
# neighbor (x,y) vectors, locating NN1,NN2,NNN1,NNN2
const neighborVectors = [(1,0),(0,1),(1,-1),(1,1)];

In [8]:
# container for lattice properties
type lattice
    # size of lattice, number of sites
    Lx::Int64;
    Ly::Int64;
    N::Int64;
    nbrs::Array{Array{Int64,1},1};
    
    function lattice(Lx::Int64,Ly::Int64,neighborVectors::Array{Tuple{Int64,Int64},1})
        neighborsList = [neighbors(site,neighborVectors,Lx,Ly) for site in 1:N];
        new(Lx,Ly,Lx*Ly,neighborsList);
    end;
end;

In [9]:
const l = lattice(Lx,Ly,neighborVectors);

In [10]:
# plaquette sites corresponding to a given site in the bottom left of the plaquette
# NOTE: there is no need to compute (store) this with a seperate function (table), the plaquette spins are given by:
#
# p1 = site, p2,p3 = NN[site], p4 = NNN[site][2]
#
# p3--p4
# |   |
# p1--p2

## Build $(S_z,k_x,k_y)$ Symmetry Sector Basis
Note: see Sandvik's Computational Studies of Quantum Spin Systems

#### specify symmetry sector

In [11]:
# choose Sz sector by specifying number of 1s in basis states
const n1 = convert(Int,N/2);
# choose kx,ky by specifying mi such that ki=2*pi*mi/Li where i=x,y ... mi is in 0:Li-1
const mx = 0;
const my = 0;
const kx = 2*pi*mx/Lx;
const ky = 2*pi*my/Ly;

In [12]:
# container for symmetry sector
type sector
    # number of 1 bits in the bit rep. of spin states in this sector
    n1::Int64;
    # momentum
    kx::Float64;
    ky::Float64;
end;

In [13]:
const s = sector(n1,kx,ky);

#### defining some useful functions for this section

In [14]:
# map integer to array of spins
function intToSpins(i::Int64,l::lattice)
    return digits(Int8,i,2,l.N)::Array{Int8,1};
end;

# map array of spins to integer
function spinsToInt(spins::Array{Int8,1})
    i::Int64 = 0;
    for (ii,s) in enumerate(spins)
        i += s*2^(ii-1);
    end;
    return i;
end;

In [15]:
# x translation operator
function Tx!(spins::Array{Int8,1},l::lattice)
    # loop over rows of the lattice
    for y::Int in 0:l.Ly-1
        # store last element of the row
        temp::Int8 = spins[(y+1)*l.Lx];
        # roll the row
        for x::Int in l.Lx-1:-1:1
            spins[y*l.Lx+1+x] = spins[y*l.Lx+x];
        end;
        # take care of the first element
        spins[y*l.Lx+1] = temp;
    end;
    
    return spins;
end;

# y translation operator
function Ty!(spins::Array{Int8,1},l::lattice)
    # store the top row
    temp::Array{Int,1} = spins[(l.Ly-1)*l.Lx+1:l.Ly*l.Lx];
    # loop over rows of the lattice
    for y::Int in l.Ly-1:-1:1
        spins[y*l.Lx+1:(y+1)*l.Lx] = spins[(y-1)*l.Lx+1:y*l.Lx];
    end;
    # replace bottom row with initial value of top row
    spins[1:l.Lx] = temp;
    
    return spins;
end;

In [16]:
# function for computing the normalization constant Na such that the momentum state is 1/sqrt(Na) * sum(phase * TxTy |b>)
function normConstant(b::Int64,l::lattice,s::sector)
    # spin config corresponding to b
    spins::Array{Int8,1} = intToSpins(b,l);
    
    # sum of phases
    F::Complex128 = 0.0+0.0im;
    # normalization constant
    Na::Float64 = 0.0;

    # the translated state in integer rep.
    Tb::Int64 = b;
    # set of unique translated states in integer rep.
    Tbs::Set{Int64} = Set{Int64}();
    
    
    # perform all translations
    for y in 0:l.Ly-1
        for x in 0:l.Lx-1
            # add to set of translated states
            Tb = spinsToInt(spins);
            push!(Tbs,Tb);
            
            if Tb == b
                # add to sum of phases
                F += exp(-1.0im*(s.kx*x+s.ky*y));
            elseif Tb < b
                return 0.0
            end;
            
            Tx!(spins,l);

        end;
        
        Ty!(spins,l);
        
    end;
    
    # compute and return normalization constant
    Na = abs2(F)*length(Tbs);
    
    return Na;
end;

#### construct the basis

In [17]:
type SzkxkyBasis
    # list of representatives of momentum basis states in integer representation
    b::Array{Int64};
    # list of corresponding normalization constants
    n::Array{Float64};
    # dimension of Hilbert space
    dim::Int64;
    
    # constructor
    function SzkxkyBasis(l::lattice,s::sector)
        # initialize list of reps of momentum basis elements
        bList::Array{Int64,1} = Int64[];
        # initialize list of normalization constants
        nList::Array{Float64,1} = Float64[];

        # run up the binary odometer of Sz states
        # start with first n1 bits in state 1(down)
        b::Int64 = 2^n1-1;
        sb::Array{Int8,1} = intToSpins(b,l);

        # useful Int8 versions of 0 and 1
        Int80 = convert(Int8,0);
        Int81 = convert(Int8,1);

        while true
            # check if this state meets the required kx,ky
            n = normConstant(b,l,s);

            # if valid rep state, add info to basis
            if n > 0.0
                push!(bList,b);
                push!(nList,n);
            end;

            # counter of 1 bits to the right (in binary convention)
            i::Int = 0;
            # position in bit array
            j::Int = 1;
            while j < l.N
                # find a 1 bit whose following neighbor is 0
                if sb[j] == Int81
                    if sb[j+1] == Int81
                        i += 1;
                    else
                        sb[1:i] = Int81;
                        sb[i+1:j] = Int80;
                        sb[j+1] = Int81;
                        break
                    end;
                end;
                # to the next bit
                j += 1;
            end;

            # if all 1s got shifted completely to the left, then all states are explored
            if j == l.N
                break
            end;

            # update integer rep. of basis state
            b = spinsToInt(sb);
        end;
        
        # construct basis
        new(bList,nList,length(bList));
    end;
    
end;

In [18]:
# construct the Sz,kx,ky sector basis
# 38.6 seconds for 6x4 lattice
@time const basis = SzkxkyBasis(l,s);
print("Dimension of reduced Hilbert space is ",basis.dim,".");

#### some more useful functions for working with this basis

In [19]:
# search ordered basis for index of integer representation of spin state
function basisIndex(basisElement::Int64,basis::SzkxkyBasis)
    bIndex::UnitRange{Int64} = searchsorted(basis.b,basisElement);
    if !isempty(bIndex)
        return bIndex[1]::Int64;
    else
        return 0::Int64;
    end;
end;

In [20]:
# find related representative state and what translation relates the two states
function representative(b::Int64,l::lattice)
    # bit array
    spins::Array{Int8,1} = intToSpins(b,l);
    # rep. state
    rep::Int64 = b;
    # translated state
    Tb::Int64 = b;
    # translations to get to rep. state
    lx::Int64 = 0;
    ly::Int64 = 0;
    
    # perform all translations
    for y::Int in 0:l.Ly
        for x::Int in 0:l.Lx
            # translated state
            Tb = spinsToInt(spins);
            
            if Tb < b
                # then this is a better representative
                rep = Tb;
                lx = x;
                ly = y;
            end;
            Tx!(spins,l);
        end;
        Ty!(spins,l);
    end;
    
    return rep,lx,ly;
end;

## Sparse Hamiltonian
$H=\frac{1}{4} \left( J_1 \sum_{\langle ij \rangle} + J_2 \sum_{\langle \langle ij \rangle \rangle} \right) \bf{\sigma}_i \cdot \bf{\sigma}_j + \frac{1}{8} K \sum_{p} \left( \bf{\sigma}_{p_1} \cdot \bf{\sigma}_{p_2} \right) \left( \bf{\sigma}_{p_3} \cdot \bf{\sigma}_{p_4} \right) + \left( \bf{\sigma}_{p_1} \cdot \bf{\sigma}_{p_3} \right) \left( \bf{\sigma}_{p_2} \cdot \bf{\sigma}_{p_4} \right) - \left( \bf{\sigma}_{p_1} \cdot \bf{\sigma}_{p_4} \right) \left( \bf{\sigma}_{p_2} \cdot \bf{\sigma}_{p_3} \right)$

#### useful functions for manipulating states

In [21]:
# function for flipping two spins of a basis state
function XiXj(b::Int64,i::Int64,j::Int64)
    # b - integer rep of state
    # i,j - sites of spin flips
    
    return (2^(i-1) + 2^(j-1)) $ b;
end;

# function for flipping four spins of a basis state
function XiXjXkXl(b::Int64,i::Int,j::Int,k::Int,l::Int)
    # b - integer rep of state
    # i,j,k,l - sites of spin flips
    
    return (2^(i-1) + 2^(j-1) + 2^(k-1) + 2^(l-1)) $ b;
end;

#### functions for calculating matrix elements

In [22]:
# function for ZZ diagonal matrix element (same in spin basis and momentum basis)
function HbZZb(sb::Array{Int8,1},i::Int64,nbrs::Array{Int64,1},c::couplings)
    # sb - spin rep. of basis state
    # i - site on lattice being looped over
    # nbrs - list of neighbors of site i connected by ZZ bond
    
    # compute matrix element
    # J1 terms
    matElem::Complex128 = 0.25*c.J1*(-1)^(sb[i])*((-1)^(sb[nbrs[1]])+(-1)^(sb[nbrs[2]])) + 0.0im;
    # J2 terms
    matElem += 0.25*c.J2*(-1)^(sb[i])*((-1)^(sb[nbrs[3]])+(-1)^(sb[nbrs[4]]));
    
    return matElem;
end;

In [23]:
# function for off-diagonal XX+YY matrix elements in momentum basis
function HbXXpYYa(b::Int64,bIndex::Int64,i::Int64,j::Int64,J::Float64,basis::SzkxkyBasis,s::sector,l::lattice)
    # bIndex - index of b in the basis
    # i,j - sites of XXpYY operator
    # J - J1 or J2 given
    
    # integer rep. of X_i X_j |b>
    a::Int64 = XiXj(b,i,j);
    # representative of state |a>
    repa::Int64,lx::Int64,ly::Int64 = representative(a,l);
    # index of rep. of state |a>
    aIndex = basisIndex(repa,basis);
    
    # if not in the reduced basis, matrix element will be zero
    if aIndex > 0
        return aIndex,((0.5*J)*exp(-1.0im*(s.kx*lx+s.ky*ly))*sqrt(basis.n[aIndex]/basis.n[bIndex])+0.0im)::Complex128;
    else
        return 0,0.0+0.0im;
    end;   
end;

In [24]:
# function for all XXXX plaquette terms in momentum basis
function HbXXpYYtXXpYYa(b::Int64,bIndex::Int64,p::Array{Int,1},sp::Array{Int8,1},basis::SzkxkyBasis,c::couplings,s::sector,l::lattice)
    # integer rep. of X_i X_j X_k X_l |b>
    a::Int64 = XiXjXkXl(b,p[1],p[2],p[3],p[4]);
    # representative of |a>
    repa::Int64,lx::Int64,ly::Int64 = representative(a,l);
    # index of rep. of state |a>
    aIndex = basisIndex(repa,basis);
    
    # a term that shows up in all matrix elements
    plaqSign::Int = (-1)^(sum(sp));
    
    # compute matrix element in spin basis, three (S.S)(S.S) terms
    preMatEl::Int = 1 + plaqSign;
    preMatEl += -(-1)^(sp[1]+sp[2])-(-1)^(sp[3]+sp[4]);
    preMatEl += -(-1)^(sp[1]+sp[3])-(-1)^(sp[2]+sp[4]);
    preMatEl -= -(-1)^(sp[1]+sp[4])-(-1)^(sp[2]+sp[3]);
    
    # return momentum basis index of bra and matrix element
    if aIndex > 0
        return aIndex,((0.125*c.K*preMatEl)*exp(-1.0im*(s.kx*lx+s.ky*ly))*sqrt(basis.n[aIndex]/basis.n[bIndex])+0.0im)::Complex128;
    else
        return 0,0.0+0.0im;
    end;
end;

In [25]:
# function for all ZZZZ plaquette terms (same in spin and momentum basis')
function HbZZZZb(sp::Array{Int8,1},c::couplings)
    return ((0.125*c.K*(-1)^(sum(sp)))+0.0im)::Complex128;
end;

In [26]:
# function for (XX+YY)ZZ plaq. terms ex: (XiXj+YiYj)ZkZl i.e. which reduced basis states (and with which matrix elements)
# are connected by the Hamiltonian to state b, **momentum basis
function HbXXpYYZZa(b::Int64,bIndex::Int64,p::Array{Int,1},sp::Array{Int8,1},basis::SzkxkyBasis,c::couplings,s::sector,l::lattice)
    # p - a list of plaquette sites [pi,pj,pk,pl] in standard lattice order [bottom left, bottom right, top left, top right]
    # sp - a list of plaquette spins [si,sj,sk,sl]
    # K - coupling constant
    
    # a term that shows up in all matrix elements
    plaqSign::Int = (-1)^(sum(sp));
    
    # reduced momentum basis indices and matrix elements for operators [XiXj,XkXl]
    inds::Array{Int64,1} = Int64[];
    matEls::Array{Complex128,1} = Complex128[];
    
    # fill with states and matrix elements
    # t is (flip spin,flip spin,other spin,other spin,sign of matrix element)
    for t in [(1,2,3,4,1),(3,4,1,2,1),(1,3,2,4,1),(2,4,1,3,1),(1,4,2,3,-1),(2,3,1,4,-1)]
        # representative of two-spin-flipped |b>
        repa::Int64,lx::Int64,ly::Int64 = representative(XiXj(b,p[t[1]],p[t[2]]),l);
        # index of this representative in basis
        aIndex::Int64 = basisIndex(repa,basis);
        # compute matrix element
        matEl::Complex128 = 0.0+0.0im;
        if aIndex > 0
            matEl = (t[5]*0.125*c.K*((-1)^(sp[t[3]]+sp[t[4]])-plaqSign)*exp(-1.0im*(s.kx*lx+s.ky*ly))*sqrt(basis.n[aIndex]/basis.n[bIndex])+0.0im)::Complex128;
        end;
        push!(inds,aIndex);
        push!(matEls,matEl);
    end;
    
    return inds,matEls;
end;

#### construct the sparse Hamiltonian

In [27]:
# function for building the sparse Hamiltonian
function constructSparseHam(basis::SzkxkyBasis,c::couplings,s::sector,l::lattice)
    # store location and value of non-zero matrix elements
    I::Array{Int64} = Int64[];
    J::Array{Int64} = Int64[];
    M::Array{Complex128} = Complex128[];
    
    for bIndex::Int64 in 1:basis.dim
        
        b::Int64 = basis.b[bIndex]; # integer rep.
        sb::Array{Int8,1} = intToSpins(b,l); # spin rep.

        # run over operators in the Hamiltonian at each site of the lattice
        for site::Int64 in 1:N
            
            # neighbors (having J1 J2 bonds to this site)
            NN1::Int,NN2::Int,NNN1::Int,NNN2::Int = l.nbrs[site];
            # plaquette p1,p2,p3,p3
            p::Array{Int64,1} = [site,NN1,NN2,NNN2];
            # spins of the plaquette
            sp::Array{Int8,1} = [sb[pi] for pi in p];

            
            # 1) diagonal matrix element

            # compute matrix element from J1 J2 K terms
            diagMatEl::Complex128 = HbZZb(sb,site,l.nbrs[site],c) + HbZZZZb(sp,c) + 0.0im;

            if diagMatEl != 0.0+0.0im
                # fill H
                push!(I,bIndex);
                push!(J,bIndex);
                push!(M,diagMatEl);
                
            end;

            
            # 2) off diagonal matrix elements

            # NN1 & NN2 XX+YY J1 bonds
            for nbr::Int64 in [NN1,NN2]
                reducedXXbNN::Int64,offDiagHNN::Complex128 = HbXXpYYa(b,bIndex,site,nbr,J1,basis,s,l);

                if reducedXXbNN > 0 && offDiagHNN != 0.0+0.0im
                    # fill H
                    push!(I,bIndex);
                    push!(J,reducedXXbNN);
                    push!(M,offDiagHNN);
                end;
            end;
            
            # NNN1 & NNN2 XX+YY J2 bonds
            for nbr::Int in [NNN1,NNN2]
                reducedXXbNNN,offDiagHNNN = HbXXpYYa(b,bIndex,site,nbr,J2,basis,s,l);

                if reducedXXbNNN > 0 && offDiagHNNN != 0.0+0.0im
                    # fill H
                    push!(I,bIndex);
                    push!(J,reducedXXbNNN);
                    push!(M,offDiagHNNN);
                end;
            end;
            
            
            # 3) ring exchange terms
            
            # XXXX term
            reducedXXXXb::Int,matElemXXXX::Complex128 = HbXXpYYtXXpYYa(b,bIndex,p,sp,basis,c,s,l);
            
            if reducedXXXXb > 0 && matElemXXXX != 0.0+0.0im
                # fill H
                push!(I,bIndex);
                push!(J,reducedXXXXb);
                push!(M,matElemXXXX);
            end;
            
            # (XX+YY)ZZ terms
            raList::Array{Int64,1},matElList::Array{Complex128,1} = HbXXpYYZZa(b,bIndex,p,sp,basis,c,s,l);
            
            # loop over connected reduced states in raList
            for ii::Int in 1:length(raList)
                if raList[ii] > 0 && matElList[ii] != 0.0+0.0im
                    # fill H
                    push!(I,bIndex);
                    push!(J,raList[ii]);
                    push!(M,matElList[ii]);
                end;
            end;
        end;
    end;
    
    H::SparseMatrixCSC{Complex,Int64} = sparse(I,J,M,basis.dim,basis.dim);
    return H;
end;

In [28]:
# build the sparse Hamiltonian
# takes 700 seconds for 6x4 J1-only Heisenberg model
@time const H = constructSparseHam(basis,c,s,l);

  3.235533 seconds (6.36 M allocations: 467.113 MB, 1.37% gc time)


## Compute Eigenvalues

In [29]:
# parameters for eigenvalue calculation
numEigs = 4;
tolerance = 10.^(-5.);
ritzVec = false;
numKrylovVecs = 20;
maxIter = 300;

In [30]:
# ~6.71 seconds for 6x4, numEigs=4, which=:SR, J1 Heisenberg Hamiltonian
@time eigsResult = eigs(H; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec);  #:LM stands for largest magnitude, :SR for smallest real part
;

  3.255930 seconds (4.00 M allocations: 161.671 MB, 1.82% gc time)


In [31]:
# energies
print(real(eigsResult[1]));

[-8.96981,-8.06103,-6.7027,-6.67496]

In [32]:
# algorithm performance
print("Number of iterations = ",eigsResult[3],"\n");
print("Number of matrix-vector multiplications = ",eigsResult[4],"\n");

Number of iterations = 3
Number of matrix-vector multiplications = 50
